### Conversion Pipeline
```python
#Initialize SnnTorch/SpikingJelly model
net = SNN(T=num_steps)

#Train the SNN
...

#Fold the BN layer 
bn = BN_Folder() 
net_bn = bn.fold(net)

#Weight, Bias Quantization 
qn = Quantize_Network() 
net_quan = qn.quantize(net_bn)

#Convert to HiAER-Spike Dictionaries
num_steps = 4
input_layer = 0
output_layer = 11
input_size = (3, 32, 32)
backend = 'snnTorch'
threshold = qn.v_threshold

cn = CRI_Converter(num_steps = num_steps, input_layer = input_layer, output_layer = output_layer, input_shape = input_shape backend=backend, v_threshold = v_threshold)
cn.layer_converter(net_quan)

```

In [ ]:
### Training SNN with spikingjelly
from spikingjelly.activation_based import neuron, layer, surrogate_function

class SNN(nn.module):
    def __init__(self, tau = 2.0):
        self.flatten = layer.Flatten()
        self.linear = layer.Linear(28 * 28, 10, bias=False),
        self.lif = neuron.LIFNode(tau=tau, surrogate_function=surrogate.ATan())
    def forward(self, x):
        x = self.flatten(x)
        x = self.linear(x)
        x = self.lif(x)
        return x

# Prepare the dataset
mnist_train = datasets.MNIST(args.data_path, train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]))
mnist_test = datasets.MNIST(args.data_path, train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]))

...

### Using command line to train a 2-layer feedforward model 
python HiAER_demo.py --train -b 128 -T 4 --lr 1e-1 --epoch 20 --encoder

        

### Running it on HiAER-Spike 

```python
#Define a configuration dictionary
config = {}
config['neuron_type'] = "I&F"
config['global_neuron_params'] = {}
config['global_neuron_params']['v_thr'] = qn.v_threshold

#Initialize a HiAER-Spike network
hardwareNetwork = CRI_network(axons=cn.axon_dict,connections=cn.neuron_dict,config=config,target='CRI', outputs = cn.output_neurons, coreID=1)
softwareNetwork = CRI_network(axons=cn.axon_dict,connections=cn.neuron_dict,config=config, target='simpleSim', outputs = cn.output_neurons)

#Run the HiAER-Spike network on test dataset
cn.run(hardwareNetwork, loss_func, test_loader, 'hardware')
cn.run(softwareNetwork, loss_func, test_loader, 'software')    

```

In [4]:
import numpy as np

def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', 10)
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value
a = np.arange(6)
a = a.reshape((2, 3))
np.pad(a, 1, pad_with, padder=-1)

array([[-1, -1, -1, -1, -1],
       [-1,  0,  1,  2, -1],
       [-1,  3,  4,  5, -1],
       [-1, -1, -1, -1, -1]])